In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import keras
from keras.applications import VGG16, InceptionV3
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model, load_model, save_model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

from google.colab import drive

from IPython.core.display import display

from sklearn.model_selection import train_test_split

import os
import shutil


drive_mount = os.path.abspath('./gdrive')
base_dir = os.path.join(drive_mount, 'My Drive/assignment_5')
models_dir = os.path.join(base_dir, 'saved_models')


# mount gdrive and unzip data

drive.mount(drive_mount, force_remount=True)
!unzip -qo "./gdrive/My Drive/assignment_5/hvc_data.zip"
print(os.listdir(base_dir))

# load annotations

df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
display(df)

# one hot encoding of labels

one_hot_df = pd.concat(
    [
        df[["image_path"]],
        pd.get_dummies(df.gender, prefix="gender"),
        pd.get_dummies(df.imagequality, prefix="imagequality"),
        pd.get_dummies(df.age, prefix="age"),
        pd.get_dummies(df.weight, prefix="weight"),
        pd.get_dummies(df.carryingbag, prefix="carryingbag"),
        pd.get_dummies(df.footwear, prefix="footwear"),
        pd.get_dummies(df.emotion, prefix="emotion"),
        pd.get_dummies(df.bodypose, prefix="bodypose"),
    ], 
    axis = 1
)

display(one_hot_df)


# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    def __init__(self, df, batch_size=32, shuffle=True, augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        
        images = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])        
        if self.augmentation is not None:
            images = self.augmentation.flow(images, shuffle=False).next()
        
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        
        return images, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

train_df, val_df = train_test_split(one_hot_df, test_size=0.15)

print(train_df.shape, val_df.shape)
display(train_df)

# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32)
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)

# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
print('no. of units:', num_units)

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255):
    def eraser(input_img):
        img_h, img_w, _ = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        c = np.random.uniform(v_l, v_h)
        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser
    
def build_model():
    backbone = InceptionV3(
       include_top=False, 
       weights=None, # Random initialization of weights
       input_tensor=Input(shape=(224, 224, 3)),
    )

    neck = backbone.output
    neck = Flatten(name="flatten")(neck)
    neck = Dense(512, activation="relu")(neck)

    def build_tower(in_layer):
        neck = Dropout(0.3)(in_layer)
        neck = Dense(128, activation="relu")(neck)
        return neck

    def build_head(name, in_layer):
        return Dense(
            num_units[name], activation="softmax", name=f"{name}_output"
        )(in_layer)

    # heads
    gender = build_head("gender", build_tower(neck))
    image_quality = build_head("image_quality", build_tower(neck))
    age = build_head("age", build_tower(neck))
    weight = build_head("weight", build_tower(neck))
    bag = build_head("bag", build_tower(neck))
    footwear = build_head("footwear", build_tower(neck))
    emotion = build_head("emotion", build_tower(neck))
    pose = build_head("pose", build_tower(neck))

    model = Model(
        inputs=backbone.input, 
        outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion],
    )

    compile_model(model)
    model.summary()
    
    return model

def compile_model(model, *, lr=1e-3, **kwargs):
    model.compile(
        optimizer=SGD(lr=lr, momentum=0.9),
        loss="categorical_crossentropy", 
        metrics=["accuracy"],
        **kwargs,
    )    
    
def load_saved_model(dir_name):
    model_dir = os.path.join(models_dir, dir_name)
    
    last_model = max(os.listdir(model_dir))
    last_model = os.path.join(model_dir, last_model)
    
    print(f'Loading model from {last_model!r}...')
    model = load_model(last_model)

    return model

def train_model(model, dir_name, epochs=50, callbacks=[]):    
    model_dir = os.path.join(models_dir, dir_name)
    shutil.rmtree(model_dir, ignore_errors=True)
    os.makedirs(model_dir)

    model.fit_generator(
        generator=train_gen,
        validation_data=valid_gen,
        use_multiprocessing=True,
        workers=6, 
        epochs=epochs,
        verbose=2,
        callbacks=[
            ModelCheckpoint(
                os.path.join(model_dir, 'model_{epoch:03d}.hdf5'),
                save_best_only=True,
                verbose=1,
            ),
            *callbacks,
        ],
    )

    final_model_filepath = os.path.join(model_dir, f'model_{epochs}.hdf5')
    save_model(model, final_model_filepath)


def evaluate_model(model):
    results = model.evaluate_generator(valid_gen, verbose=1)
    accuracies = {}
    losses = {}
    for k, v in zip(model.metrics_names, results):
        if k.endswith('acc'):
            accuracies[k] = round(v * 100, 4) 
        else:
            losses[k] = v
    return accuracies, losses

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
['hvc_data.zip', 'saved_models']


,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg
...,...,...,...,...,...,...,...,...,...
13568,male,Average,35-45,slightly-overweight,None,Normal,Happy,Front-Frontish,resized/13570.jpg
13569,female,Average,25-35,normal-healthy,Daily/Office/Work Bag,Fancy,Neutral,Front-Frontish,resized/13571.jpg
13570,female,Bad,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Side,resized/13572.jpg
13571,female,Bad,25-35,over-weight,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/13573.jpg


,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
0,resized/1.jpg,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0
1,resized/2.jpg,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0
2,resized/3.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0
3,resized/4.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
4,resized/5.jpg,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13568,resized/13570.jpg,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0
13569,resized/13571.jpg,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0
13570,resized/13572.jpg,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1
13571,resized/13573.jpg,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0


(11537, 28) (2036, 28)


,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
4290,resized/4291.jpg,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0
4615,resized/4616.jpg,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0
8272,resized/8273.jpg,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,1,0
3960,resized/3961.jpg,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1
1704,resized/1705.jpg,0,1,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8672,resized/8673.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0
3535,resized/3536.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0
10367,resized/10369.jpg,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0
6516,resized/6517.jpg,1,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0


no. of units: {'gender': 2, 'image_quality': 3, 'age': 5, 'weight': 4, 'bag': 3, 'pose': 3, 'footwear': 3, 'emotion': 4}


In [4]:
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
)

model = load_saved_model('s16')

K.set_value(model.optimizer.lr, 1e-5)
train_model(model, 's17')

model = load_saved_model('s17')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s16/model_001.hdf5'...
Epoch 1/50

 - 95s - loss: 2.1762 - gender_output_loss: 0.0790 - image_quality_output_loss: 0.4034 - age_output_loss: 0.4817 - weight_output_loss: 0.3533 - bag_output_loss: 0.2034 - footwear_output_loss: 0.2150 - pose_output_loss: 0.1067 - emotion_output_loss: 0.3337 - gender_output_acc: 0.9654 - image_quality_output_acc: 0.8396 - age_output_acc: 0.8084 - weight_output_acc: 0.8635 - bag_output_acc: 0.9224 - footwear_output_acc: 0.9179 - pose_output_acc: 0.9613 - emotion_output_acc: 0.8649 - val_loss: 1.2301 - val_gender_output_loss: 0.0220 - val_image_quality_output_loss: 0.2349 - val_age_output_loss: 0.2800 - val_weight_output_loss: 0.2092 - val_bag_output_loss: 0.0908 - val_footwear_output_loss: 0.1182 - val_pose_output_loss: 0.0395 - val_emotion_output_loss: 0.2355 - val_gender_output_acc: 0.9909 - val_image_quality_output_acc: 0.9546 - val_age_output_acc: 0.9370 - val_weight_output_acc: 0.

({'age_output_acc': 91.6835,
  'bag_output_acc': 97.5302,
  'emotion_output_acc': 90.5746,
  'footwear_output_acc': 96.6734,
  'gender_output_acc': 98.8911,
  'image_quality_output_acc': 94.8085,
  'pose_output_acc': 98.8407,
  'weight_output_acc': 93.498},
 {'age_output_loss': 0.2912717673086351,
  'bag_output_loss': 0.1167130374139355,
  'emotion_output_loss': 0.24948640408054476,
  'footwear_output_loss': 0.14105815344279812,
  'gender_output_loss': 0.022075867945057972,
  'image_quality_output_loss': 0.2314601117564786,
  'loss': 1.3007311090346305,
  'pose_output_loss': 0.0516795031576147,
  'weight_output_loss': 0.19698628422714048})

In [0]:
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
)

model = load_saved_model('s16')

K.set_value(model.optimizer.lr, 1e-3)
train_model(model, 's17')

model = load_saved_model('s17')
evaluate_model(model)

In [0]:
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
)

model = load_saved_model('s16')

K.set_value(model.optimizer.lr, 1e-4)
train_model(model, 's17')

model = load_saved_model('s17')
evaluate_model(model)

In [0]:
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
)

model = load_saved_model('s16')

K.set_value(model.optimizer.lr, 1e-5)
train_model(model, 's17')

model = load_saved_model('s17')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s16/model_001.hdf5'...
Epoch 1/50


 - 101s - loss: 2.1769 - gender_output_loss: 0.0773 - image_quality_output_loss: 0.4087 - age_output_loss: 0.4746 - weight_output_loss: 0.3565 - bag_output_loss: 0.2043 - footwear_output_loss: 0.2172 - pose_output_loss: 0.1082 - emotion_output_loss: 0.3302 - gender_output_acc: 0.9649 - image_quality_output_acc: 0.8404 - age_output_acc: 0.8207 - weight_output_acc: 0.8626 - bag_output_acc: 0.9207 - footwear_output_acc: 0.9195 - pose_output_acc: 0.9608 - emotion_output_acc: 0.8672 - val_loss: 1.2578 - val_gender_output_loss: 0.0248 - val_image_quality_output_loss: 0.2371 - val_age_output_loss: 0.2842 - val_weight_output_loss: 0.2117 - val_bag_output_loss: 0.0957 - val_footwear_output_loss: 0.1205 - val_pose_output_loss: 0.0446 - val_emotion_output_loss: 0.2393 - val_gender_output_acc: 0.9879 - val_image_quality_output_acc: 0.9567 - val_age_output_acc: 0.9370 - val_weight_output_acc: 

In [0]:
# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
    augmentation=ImageDataGenerator(
        rotation_range=45,
    )
)

# load saved model and continue
model = load_saved_model('s15')

# reset LR
K.set_value(model.optimizer.lr, 1e-6)

# save last model
train_model(model, 's16', 50)

# evaluate best model
model = load_saved_model('s16')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s15/model_014.hdf5'...













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
 - 103s - loss: 10.2771 - gender_output_loss: 0.8815 - image_quality_output_loss: 1.0700 - age_output_loss: 1.6627 - weight_output_loss: 1.1067 - bag_output_loss: 1.1426 - footwear_output_loss: 1.8477 - pose_output_loss: 1.4451 - emotion_output_loss: 1.1208 - gender_output_acc: 0.7322 - image_quality_output_acc: 0.5639 - age_output_acc: 0.4451 - weight_output_acc: 0.6293 - bag_output_acc: 0.6372 - footwear_output_acc: 0.5926 - pose_output_acc: 0.6332 - emotion_output_acc: 0.6851 - val_loss: 1.6074 - val_gender_output_loss: 0.0396 - val_image_quality_output_loss: 0.3163 - val_age_output_loss: 0.3589 - val_weight_output_loss: 0.2534 - val_bag_output_loss: 0.1261 

({'age_output_acc': 86.9456,
  'bag_output_acc': 94.3044,
  'emotion_output_acc': 85.6855,
  'footwear_output_acc': 93.8508,
  'gender_output_acc': 97.9839,
  'image_quality_output_acc': 87.4496,
  'pose_output_acc': 96.2198,
  'weight_output_acc': 88.256},
 {'age_output_loss': 0.5679545075662674,
  'bag_output_loss': 0.23441922087823192,
  'emotion_output_loss': 0.38806576209683574,
  'footwear_output_loss': 0.26138744719566837,
  'gender_output_loss': 0.12047455375713687,
  'image_quality_output_loss': 0.4713536424021567,
  'loss': 2.5825602046905027,
  'pose_output_loss': 0.16307590661510343,
  'weight_output_loss': 0.37582914675435714})

In [0]:
# load saved model and continue
model = load_saved_model('s14')

# decrease LR
K.set_value(model.optimizer.lr, 1e-7)

train_model(model, 's15', 100)

# evaluate best model
model = load_saved_model('s15')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s14/model_005.hdf5'...
Epoch 1/100
 - 102s - loss: 2.1486 - gender_output_loss: 0.0726 - image_quality_output_loss: 0.4152 - age_output_loss: 0.4684 - weight_output_loss: 0.3511 - bag_output_loss: 0.1987 - footwear_output_loss: 0.2066 - pose_output_loss: 0.1035 - emotion_output_loss: 0.3326 - gender_output_acc: 0.9654 - image_quality_output_acc: 0.8365 - age_output_acc: 0.8147 - weight_output_acc: 0.8611 - bag_output_acc: 0.9207 - footwear_output_acc: 0.9234 - pose_output_acc: 0.9631 - emotion_output_acc: 0.8644 - val_loss: 1.2157 - val_gender_output_loss: 0.0257 - val_image_quality_output_loss: 0.2321 - val_age_output_loss: 0.2861 - val_weight_output_loss: 0.1989 - val_bag_output_loss: 0.0907 - val_footwear_output_loss: 0.1213 - val_pose_output_loss: 0.0373 - val_emotion_output_loss: 0.2236 - val_gender_output_acc: 0.9884 - val_image_quality_output_acc: 0.9536 - val_age_output_acc: 0.9380 - val_weight_output_acc: 0

({'age_output_acc': 93.8004,
  'bag_output_acc': 97.8327,
  'emotion_output_acc': 92.0867,
  'footwear_output_acc': 96.8246,
  'gender_output_acc': 98.9919,
  'image_quality_output_acc': 95.4133,
  'pose_output_acc': 98.9919,
  'weight_output_acc': 95.0101},
 {'age_output_loss': 0.2838474786089313,
  'bag_output_loss': 0.08996555713876601,
  'emotion_output_loss': 0.22227796864125035,
  'footwear_output_loss': 0.11781493430176089,
  'gender_output_loss': 0.025879984774116065,
  'image_quality_output_loss': 0.2296616747494667,
  'loss': 1.2043233763787053,
  'pose_output_loss': 0.03725467802536103,
  'weight_output_loss': 0.19762110806280567})

In [0]:
# load saved model and continue
model = load_saved_model('s13')

# decrease LR
K.set_value(model.optimizer.lr, 1e-6)

train_model(model, 's14', 100)

# evaluate best model
model = load_saved_model('s14')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s13/model_053.hdf5'...
Epoch 1/100
Epoch 1/100
 - 96s - loss: 2.1348 - gender_output_loss: 0.0753 - image_quality_output_loss: 0.3976 - age_output_loss: 0.4722 - weight_output_loss: 0.3496 - bag_output_loss: 0.1981 - footwear_output_loss: 0.2095 - pose_output_loss: 0.1047 - emotion_output_loss: 0.3278 - gender_output_acc: 0.9639 - image_quality_output_acc: 0.8396 - age_output_acc: 0.8093 - weight_output_acc: 0.8589 - bag_output_acc: 0.9194 - footwear_output_acc: 0.9226 - pose_output_acc: 0.9640 - emotion_output_acc: 0.8675 - val_loss: 1.2161 - val_gender_output_loss: 0.0265 - val_image_quality_output_loss: 0.2296 - val_age_output_loss: 0.2846 - val_weight_output_loss: 0.1980 - val_bag_output_loss: 0.0908 - val_footwear_output_loss: 0.1250 - val_pose_output_loss: 0.0378 - val_emotion_output_loss: 0.2238 - val_gender_output_acc: 0.9889 - val_image_quality_output_acc: 0.9556 - val_age_output_acc: 0.9390 - val_weight_ou

Process ForkPoolWorker-309:
Process ForkPoolWorker-305:
Process ForkPoolWorker-308:
Process ForkPoolWorker-303:
Process ForkPoolWorker-302:
Process ForkPoolWorker-311:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-312:
Process ForkPoolWorker-301:
Process ForkPoolWorker-307:
Process ForkPoolWorker-310:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-304:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-306:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/mu


Epoch 00025: val_loss did not improve from 1.20640
Epoch 26/100


  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/usr/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing

KeyboardInterrupt: ignored

In [0]:
# load saved model and continue
model = load_saved_model('s12')

# decrease LR
K.set_value(model.optimizer.lr, 1e-5)

train_model(model, 's13', 100)
evaluate_model(model)

NameError: ignored

In [0]:
# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
    augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
    )
)

# load saved model and continue
model = load_saved_model('s11')

# decrease LR
K.set_value(model.optimizer.lr, 1e-5)

train_model(model, 's12', 500)
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s11/model_094.hdf5'...
Epoch 1/500
 - 92s - loss: 5.0551 - gender_output_loss: 0.3145 - image_quality_output_loss: 0.7850 - age_output_loss: 1.0018 - weight_output_loss: 0.7056 - bag_output_loss: 0.5642 - footwear_output_loss: 0.5691 - pose_output_loss: 0.4665 - emotion_output_loss: 0.6484 - gender_output_acc: 0.8442 - image_quality_output_acc: 0.6402 - age_output_acc: 0.5645 - weight_output_acc: 0.7196 - bag_output_acc: 0.7588 - footwear_output_acc: 0.7543 - pose_output_acc: 0.8147 - emotion_output_acc: 0.7624 - val_loss: 2.2863 - val_gender_output_loss: 0.0707 - val_image_quality_output_loss: 0.4760 - val_age_output_loss: 0.5285 - val_weight_output_loss: 0.3668 - val_bag_output_loss: 0.1960 - val_footwear_output_loss: 0.2283 - val_pose_output_loss: 0.0819 - val_emotion_output_loss: 0.3381 - val_gender_output_acc: 0.9768 - val_image_quality_output_acc: 0.8241 - val_age_output_acc: 0.8221 - val_weight_output_acc: 0.

Process ForkPoolWorker-4006:
Process ForkPoolWorker-4005:
Process ForkPoolWorker-4004:
Traceback (most recent call last):
Process ForkPoolWorker-4003:
Traceback (most recent call last):
Process ForkPoolWorker-3999:
Process ForkPoolWorker-4000:
Process ForkPoolWorker-4002:
Process ForkPoolWorker-4001:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-3998:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-4009:
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
Process ForkPoolWorker-4008:
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 10

KeyboardInterrupt: ignored

In [0]:
# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
    augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
    )
)

# load saved model and continue
model = load_saved_model('s10')

# decrease LR
K.set_value(model.optimizer.lr, 1e-5)

train_model(model, 's11', 100)
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s10/model_050.hdf5'...













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
360/360 [==============================] - 92s 256ms/step - loss: 5.7535 - gender_output_loss: 0.3995 - image_quality_output_loss: 0.8425 - age_output_loss: 1.1196 - weight_output_loss: 0.7874 - bag_output_loss: 0.6580 - footwear_output_loss: 0.6549 - pose_output_loss: 0.5657 - emotion_output_loss: 0.7258 - gender_output_acc: 0.7936 - image_quality_output_acc: 0.6177 - age_output_acc: 0.5152 - weight_output_acc: 0.6935 - bag_output_acc: 0.7089 - footwear_output_acc: 0.7161 - pose_output_acc: 0.7703 - emotion_output_acc: 0.7477 - val_loss: 2.4021 - val_gender_output_loss: 0.0797 - val_image_quality_output_loss: 0.4869 - val_age_output_loss: 0.5510 - val_weight_

In [0]:
# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
    augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
    )
)

# load saved model and continue
model = load_saved_model('s9')

# decrease LR
K.set_value(model.optimizer.lr, 1e-5)

train_model(model, 's10')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s9/model_047.hdf5'...
Epoch 1/50
360/360 [==============================] - 91s 254ms/step - loss: 6.0062 - gender_output_loss: 0.4336 - image_quality_output_loss: 0.8616 - age_output_loss: 1.1663 - weight_output_loss: 0.8088 - bag_output_loss: 0.6938 - footwear_output_loss: 0.6860 - pose_output_loss: 0.6122 - emotion_output_loss: 0.7438 - gender_output_acc: 0.7749 - image_quality_output_acc: 0.6089 - age_output_acc: 0.5043 - weight_output_acc: 0.6897 - bag_output_acc: 0.6910 - footwear_output_acc: 0.6967 - pose_output_acc: 0.7475 - emotion_output_acc: 0.7426 - val_loss: 2.6232 - val_gender_output_loss: 0.0900 - val_image_quality_output_loss: 0.5203 - val_age_output_loss: 0.5966 - val_weight_output_loss: 0.4300 - val_bag_output_loss: 0.2314 - val_footwear_output_loss: 0.2620 - val_pose_output_loss: 0.1098 - val_emotion_output_loss: 0.3832 - val_gender_output_acc: 0.9768 - val_image_quality_output_acc: 0.8155 - val_a

({'age_output_acc': 80.998,
  'bag_output_acc': 93.1956,
  'emotion_output_acc': 86.6935,
  'footwear_output_acc': 92.994,
  'gender_output_acc': 97.3286,
  'image_quality_output_acc': 81.3508,
  'pose_output_acc': 97.3286,
  'weight_output_acc': 86.0383},
 {'age_output_loss': 0.5723188913637592,
  'bag_output_loss': 0.21961708703348715,
  'emotion_output_loss': 0.37054845690727234,
  'footwear_output_loss': 0.2544978232153,
  'gender_output_loss': 0.08362510439849669,
  'image_quality_output_loss': 0.5114422882756879,
  'loss': 2.5323120009514595,
  'pose_output_loss': 0.10274466115140146,
  'weight_output_loss': 0.4175176865631534})

In [0]:
# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
    augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
    )
)

# load saved model and continue
model = load_saved_model('s8')

# decrease LR
K.set_value(model.optimizer.lr, 1e-5)

train_model(model, 's9')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s8/model_046.hdf5'...
Epoch 1/50
360/360 [==============================] - 89s 248ms/step - loss: 6.2702 - gender_output_loss: 0.4704 - image_quality_output_loss: 0.8755 - age_output_loss: 1.2028 - weight_output_loss: 0.8388 - bag_output_loss: 0.7319 - footwear_output_loss: 0.7282 - pose_output_loss: 0.6392 - emotion_output_loss: 0.7834 - gender_output_acc: 0.7495 - image_quality_output_acc: 0.6023 - age_output_acc: 0.4918 - weight_output_acc: 0.6751 - bag_output_acc: 0.6720 - footwear_output_acc: 0.6792 - pose_output_acc: 0.7427 - emotion_output_acc: 0.7365 - val_loss: 2.9270 - val_gender_output_loss: 0.1175 - val_image_quality_output_loss: 0.5565 - val_age_output_loss: 0.6429 - val_weight_output_loss: 0.4677 - val_bag_output_loss: 0.2868 - val_footwear_output_loss: 0.3004 - val_pose_output_loss: 0.1441 - val_emotion_output_loss: 0.4111 - val_gender_output_acc: 0.9733 - val_image_quality_output_acc: 0.7928 - val_a

{'age_output_acc': 78.9819,
 'bag_output_acc': 91.0786,
 'emotion_output_acc': 85.4335,
 'footwear_output_acc': 90.9274,
 'gender_output_acc': 97.4798,
 'image_quality_output_acc': 79.6371,
 'pose_output_acc': 96.7238,
 'weight_output_acc': 85.9375}

In [0]:
# add image augmentation
train_gen = PersonDat aGenerator(
    train_df, 
    batch_size=32, 
    augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
    )
)

# load saved model and continue
model = load_saved_model('s7')

# decrease LR
K.set_value(model.optimizer.lr, 1e-5)

train_model(model, 's8')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s7/model_050.hdf5'...
Epoch 1/50
360/360 [==============================] - 270s 749ms/step - loss: 6.4974 - gender_output_loss: 0.5006 - image_quality_output_loss: 0.8889 - age_output_loss: 1.2453 - weight_output_loss: 0.8644 - bag_output_loss: 0.7598 - footwear_output_loss: 0.7650 - pose_output_loss: 0.6786 - emotion_output_loss: 0.7948 - gender_output_acc: 0.7198 - image_quality_output_acc: 0.5977 - age_output_acc: 0.4802 - weight_output_acc: 0.6734 - bag_output_acc: 0.6545 - footwear_output_acc: 0.6661 - pose_output_acc: 0.7266 - emotion_output_acc: 0.7373 - val_loss: 3.2567 - val_gender_output_loss: 0.1482 - val_image_quality_output_loss: 0.6039 - val_age_output_loss: 0.7040 - val_weight_output_loss: 0.5057 - val_bag_output_loss: 0.3242 - val_footwear_output_loss: 0.3372 - val_pose_output_loss: 0.1701 - val_emotion_output_loss: 0.4634 - val_gender_output_acc: 0.9647 - val_image_quality_output_acc: 0.7626 - val_

In [0]:
# add image augmentation
train_gen = PersonDataGenerator(
    train_df, 
    batch_size=32, 
    augmentation=ImageDataGenerator(
        horizontal_flip=True,
        vertical_flip=True,
    )
)

# load saved model and continue
model = load_saved_model('s6')

# decrease LR
K.set_value(model.optimizer.lr, 1e-5)

train_model(model, 's7')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s6/model_042.hdf5'...













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
360/360 [==============================] - 92s 255ms/step - loss: 8.4807 - gender_output_loss: 0.8616 - image_quality_output_loss: 0.9569 - age_output_loss: 1.5278 - weight_output_loss: 1.0089 - bag_output_loss: 0.9624 - footwear_output_loss: 1.0941 - pose_output_loss: 1.0562 - emotion_output_loss: 1.0127 - gender_output_acc: 0.6658 - image_quality_output_acc: 0.5776 - age_output_acc: 0.4401 - weight_output_acc: 0.6419 - bag_output_acc: 0.6082 - footwear_output_acc: 0.5984 - pose_output_acc: 0.6666 - emotion_output_acc: 0.7010 - val_loss: 3.4139 - val_gender_output_loss: 0.1596 - val_image_quality_output_loss: 0.6118 - val_age_output_loss: 0.7371 - val_weight_ou

{'age_output_acc': 77.2681,
 'bag_output_acc': 89.9194,
 'emotion_output_acc': 83.5685,
 'footwear_output_acc': 89.5665,
 'gender_output_acc': 96.2702,
 'image_quality_output_acc': 75.6552,
 'pose_output_acc': 95.1109,
 'weight_output_acc': 81.4012}

In [0]:
# load saved model and continue

model = load_saved_model('s5')

# decrease LR
K.set_value(model.optimizer.lr, 1e-7)

train_model(model, 's6')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s5/model_047.hdf5'...
Epoch 1/50
359/360 [============================>.] - ETA: 0s - loss: 2.9303 - gender_output_loss: 0.0991 - image_quality_output_loss: 0.5842 - age_output_loss: 0.6373 - weight_output_loss: 0.4842 - bag_output_loss: 0.2858 - footwear_output_loss: 0.3034 - pose_output_loss: 0.1318 - emotion_output_loss: 0.4044 - gender_output_acc: 0.9601 - image_quality_output_acc: 0.7498 - age_output_acc: 0.7382 - weight_output_acc: 0.8107 - bag_output_acc: 0.8872 - footwear_output_acc: 0.8802 - pose_output_acc: 0.9525 - emotion_output_acc: 0.8385

360/360 [==============================] - 98s 273ms/step - loss: 2.9303 - gender_output_loss: 0.0993 - image_quality_output_loss: 0.5838 - age_output_loss: 0.6376 - weight_output_loss: 0.4841 - bag_output_loss: 0.2857 - footwear_output_loss: 0.3032 - pose_output_loss: 0.1320 - emotion_output_loss: 0.4047 - gender_output_acc: 0.9599 - image_quality_output_acc: 0.7502

{'age_output_acc': 83.2661,
 'bag_output_acc': 94.2036,
 'emotion_output_acc': 89.6169,
 'footwear_output_acc': 92.5907,
 'gender_output_acc': 98.1351,
 'image_quality_output_acc': 81.4012,
 'pose_output_acc': 98.4879,
 'weight_output_acc': 87.6512}

In [0]:
# load saved model and continue

model = load_saved_model('s4')

# decrease LR
K.set_value(model.optimizer.lr, 1e-6)

train_model(model, 's5')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s4/model_011.hdf5'...













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
360/360 [==============================] - 93s 257ms/step - loss: 3.0028 - gender_output_loss: 0.1027 - image_quality_output_loss: 0.5971 - age_output_loss: 0.6581 - weight_output_loss: 0.4938 - bag_output_loss: 0.2909 - footwear_output_loss: 0.2992 - pose_output_loss: 0.1434 - emotion_output_loss: 0.4175 - gender_output_acc: 0.9566 - image_quality_output_acc: 0.7440 - age_output_acc: 0.7317 - weight_output_acc: 0.8054 - bag_output_acc: 0.8868 - footwear_output_acc: 0.8813 - pose_output_acc: 0.9497 - emotion_output_acc: 0.8348 - val_loss: 2.2295 - val_gender_output_loss: 0.0478 - val_image_quality_output_loss: 0.5051 - val_age_output_loss: 0.5061 - val_weight_ou

{'age_output_acc': 83.4677,
 'bag_output_acc': 94.3044,
 'emotion_output_acc': 89.6169,
 'footwear_output_acc': 92.5403,
 'gender_output_acc': 98.1855,
 'image_quality_output_acc': 81.25,
 'pose_output_acc': 98.4879,
 'weight_output_acc': 87.8528}

In [0]:
# load saved model and continue

model = load_saved_model('s3')

# decrease LR
K.set_value(model.optimizer.lr, 1e-5)

train_model(model, 's4')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s3/model_008.hdf5'...
Epoch 1/50
360/360 [==============================] - 273s 758ms/step - loss: 3.1817 - gender_output_loss: 0.1173 - image_quality_output_loss: 0.6296 - age_output_loss: 0.6931 - weight_output_loss: 0.5242 - bag_output_loss: 0.3182 - footwear_output_loss: 0.3177 - pose_output_loss: 0.1553 - emotion_output_loss: 0.4263 - gender_output_acc: 0.9555 - image_quality_output_acc: 0.7309 - age_output_acc: 0.7211 - weight_output_acc: 0.7931 - bag_output_acc: 0.8748 - footwear_output_acc: 0.8758 - pose_output_acc: 0.9485 - emotion_output_acc: 0.8342 - val_loss: 2.4368 - val_gender_output_loss: 0.0585 - val_image_quality_output_loss: 0.5112 - val_age_output_loss: 0.5431 - val_weight_output_loss: 0.3789 - val_bag_output_loss: 0.2412 - val_footwear_output_loss: 0.2450 - val_pose_output_loss: 0.1052 - val_emotion_output_loss: 0.3537 - val_gender_output_acc: 0.9773 - val_image_quality_output_acc: 0.8155 - val_

In [0]:
# load saved model and continue

model = load_saved_model('s2')

# decrease LR
K.set_value(model.optimizer.lr, 1e-4)

train_model(model, 's3')
evaluate_model(model)

Loading model from '/content/gdrive/My Drive/assignment_5/saved_models/s2/model_050.hdf5'...













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/50
360/360 [==============================] - 167s 465ms/step - loss: 4.5332 - gender_output_loss: 0.2268 - image_quality_output_loss: 0.7596 - age_output_loss: 0.9563 - weight_output_loss: 0.6912 - bag_output_loss: 0.5098 - footwear_output_loss: 0.4915 - pose_output_loss: 0.2972 - emotion_output_loss: 0.6008 - gender_output_acc: 0.9154 - image_quality_output_acc: 0.6627 - age_output_acc: 0.6159 - weight_output_acc: 0.7341 - bag_output_acc: 0.8003 - footwear_output_acc: 0.8092 - pose_output_acc: 0.9022 - emotion_output_acc: 0.7839 - val_loss: 3.7800 - val_gender_output_loss: 0.1622 - val_image_quality_output_loss: 0.7000 - val_age_output_loss: 0.8380 - val_weight_o

In [0]:
# load saved model and continue

model = load_saved_model('s1')
train_model(model, 's2')
evaluate_model(model)

Loading model '/content/gdrive/My Drive/assignment_5/saved_models/s1/model_020.hdf5'...
Epoch 1/50
359/360 [============================>.] - ETA: 0s - loss: 7.2717 - gender_output_loss: 0.5486 - image_quality_output_loss: 0.9698 - age_output_loss: 1.3818 - weight_output_loss: 0.9594 - bag_output_loss: 0.8644 - footwear_output_loss: 0.8452 - pose_output_loss: 0.8124 - emotion_output_loss: 0.8903 - gender_output_acc: 0.7202 - image_quality_output_acc: 0.5543 - age_output_acc: 0.4006 - weight_output_acc: 0.6365 - bag_output_acc: 0.6085 - footwear_output_acc: 0.6153 - pose_output_acc: 0.6560 - emotion_output_acc: 0.7093

360/360 [==============================] - 168s 466ms/step - loss: 7.2725 - gender_output_loss: 0.5487 - image_quality_output_loss: 0.9696 - age_output_loss: 1.3817 - weight_output_loss: 0.9597 - bag_output_loss: 0.8649 - footwear_output_loss: 0.8456 - pose_output_loss: 0.8122 - emotion_output_loss: 0.8901 - gender_output_acc: 0.7203 - image_quality_output_acc: 0.5544 - a

{'age_output_acc': 38.4577,
 'bag_output_acc': 54.6371,
 'emotion_output_acc': 69.3044,
 'footwear_output_acc': 60.131,
 'gender_output_acc': 69.9597,
 'image_quality_output_acc': 53.4274,
 'pose_output_acc': 65.877,
 'weight_output_acc': 62.1976}

In [0]:
# train from scratch

model = build_model()        
train_model(model, 's1')
evaluate_model(model)

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_283 (Conv2D)             (None, 111, 111, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_283 (BatchN (None, 111, 111, 32) 96          conv2d_283[0][0]                 
__________________________________________________________________________________________________
activation_283 (Activation)     (None, 111, 111, 32) 0           batch_normalization_283[0][0]    
____________________________________________________________________________________________